In [1]:
import pandas as pd
import glob
path_prefix = "../../../../data/temp/parking_meters/"

In [2]:
files = glob.glob(path_prefix + "/SanDiegoData_201810*.csv")


In [3]:
frame2 = pd.DataFrame()
list_ = []
for file_ in files:
    df = pd.read_csv(file_,header=0, parse_dates=['StartDateTime', 'ExpiryTime'], infer_datetime_format=True)
    list_.append(df)
frame2 = pd.concat(list_, ignore_index=True)

frame2.head()

,MeterType,PoleSerNo,StartDateTime,ExpiryTime,Amount,TransactionType,CardNumber
0,SS,5-1817,2018-10-01 07:04:42,2018-10-01 08:14:24,0.30,Cash,NaN
1,SS,WD-500,2018-10-01 08:05:03,2018-10-01 08:53:03,1.00,Cash,NaN
2,SS,WD-500,2018-10-01 08:05:20,2018-10-01 10:05:03,1.50,Cash,NaN
3,SS,9-1208,2018-10-01 08:43:23,2018-10-01 10:07:23,1.75,Cash,NaN
4,SS,9-1208,2018-10-01 08:44:34,2018-10-01 10:21:47,0.30,Cash,NaN


In [4]:
update = frame2[['MeterType','PoleSerNo', 'Amount', 'TransactionType', 'StartDateTime', 'ExpiryTime']]
update = update.rename(columns = {
    'MeterType':'meter_type',
    'StartDateTime':'trans_start',
    'ExpiryTime':'meter_expire',
    'Amount':'trans_amt',
    'TransactionType':'pay_method',
    'StartDateTime':'trans_start',
    'PoleSerNo':'pole_id'
})

update.head()

,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SS,5-1817,0.30,Cash,2018-10-01 07:04:42,2018-10-01 08:14:24
1,SS,WD-500,1.00,Cash,2018-10-01 08:05:03,2018-10-01 08:53:03
2,SS,WD-500,1.50,Cash,2018-10-01 08:05:20,2018-10-01 10:05:03
3,SS,9-1208,1.75,Cash,2018-10-01 08:43:23,2018-10-01 10:07:23
4,SS,9-1208,0.30,Cash,2018-10-01 08:44:34,2018-10-01 10:21:47


In [5]:
update['trans_amt'] = update['trans_amt'] * 100
update['trans_amt'] = update['trans_amt'].astype(int)
update.meter_type = update.meter_type.str.upper()
update.pole_id = update.pole_id.str.upper()
update.pay_method = update.pay_method.str.upper()
update.meter_type = update.meter_type.str.extract('(SS|MS)', expand=False)
update.head()

,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SS,5-1817,30,CASH,2018-10-01 07:04:42,2018-10-01 08:14:24
1,SS,WD-500,100,CASH,2018-10-01 08:05:03,2018-10-01 08:53:03
2,SS,WD-500,150,CASH,2018-10-01 08:05:20,2018-10-01 10:05:03
3,SS,9-1208,175,CASH,2018-10-01 08:43:23,2018-10-01 10:07:23
4,SS,9-1208,30,CASH,2018-10-01 08:44:34,2018-10-01 10:21:47


In [6]:
# Construct a unique id:
update['uuid'] = update.meter_type.str.cat(update.pole_id, sep="")\
        .str.cat(update.trans_start.dt.strftime('%y%m%d%H%M%S'), sep="")\
        .str.cat(update.trans_amt.astype(str), sep="")\
        .str.replace("-", "")


In [7]:
update.head()

,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire,uuid
0,SS,5-1817,30,CASH,2018-10-01 07:04:42,2018-10-01 08:14:24,SS5181718100107044230
1,SS,WD-500,100,CASH,2018-10-01 08:05:03,2018-10-01 08:53:03,SSWD500181001080503100
2,SS,WD-500,150,CASH,2018-10-01 08:05:20,2018-10-01 10:05:03,SSWD500181001080520150
3,SS,9-1208,175,CASH,2018-10-01 08:43:23,2018-10-01 10:07:23,SS91208181001084323175
4,SS,9-1208,30,CASH,2018-10-01 08:44:34,2018-10-01 10:21:47,SS9120818100108443430


In [8]:
# Rearrange column order
cols = update.columns.tolist()
# Set last column as first
cols = cols[-1:] + cols[:-1]
update = update[cols]
update.head()

,uuid,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SS5181718100107044230,SS,5-1817,30,CASH,2018-10-01 07:04:42,2018-10-01 08:14:24
1,SSWD500181001080503100,SS,WD-500,100,CASH,2018-10-01 08:05:03,2018-10-01 08:53:03
2,SSWD500181001080520150,SS,WD-500,150,CASH,2018-10-01 08:05:20,2018-10-01 10:05:03
3,SS91208181001084323175,SS,9-1208,175,CASH,2018-10-01 08:43:23,2018-10-01 10:07:23
4,SS9120818100108443430,SS,9-1208,30,CASH,2018-10-01 08:44:34,2018-10-01 10:21:47


In [9]:
portal = pd.read_csv('../../../../data/prod/treas_parking_payments_2018_datasd.csv')

portal.head()

,uuid,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SSJ50718010100571650,SS,J-507,50,CASH,2018-01-01 00:57:16,2018-01-01 00:57:16
1,MS2400E118010101163925,MS,2-400E1,25,CASH,2018-01-01 01:16:39,2018-01-01 01:16:39
2,MS2400E118010101164225,MS,2-400E1,25,CASH,2018-01-01 01:16:42,2018-01-01 01:16:42
3,SSG40018010101274125,SS,G-400,25,CASH,2018-01-01 01:27:41,2018-01-01 01:27:41
4,SSFI4151801010137505,SS,FI-415,5,CASH,2018-01-01 01:37:50,2018-01-01 01:37:50


In [18]:
# Rearrange column order
portal_cols = portal.columns.tolist()
# Set last column as first
portal_cols = portal_cols[-1:] + portal_cols[:-1]
portal = portal[cols]
portal.head()

,uuid,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SSJ50718010100571650,SS,J-507,50,CASH,2018-01-01 00:57:16,2018-01-01 00:57:16
1,MS2400E118010101163925,MS,2-400E1,25,CASH,2018-01-01 01:16:39,2018-01-01 01:16:39
2,MS2400E118010101164225,MS,2-400E1,25,CASH,2018-01-01 01:16:42,2018-01-01 01:16:42
3,SSG40018010101274125,SS,G-400,25,CASH,2018-01-01 01:27:41,2018-01-01 01:27:41
4,SSFI4151801010137505,SS,FI-415,5,CASH,2018-01-01 01:37:50,2018-01-01 01:37:50


In [10]:
portal['trans_start'] = pd.to_datetime(portal['trans_start'],  format="%Y-%m-%d %H:%M:%S")
portal['meter_expire'] = pd.to_datetime(portal['meter_expire'],  format="%Y-%m-%d %H:%M:%S")
portal.head()

,uuid,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SSJ50718010100571650,SS,J-507,50,CASH,2018-01-01 00:57:16,2018-01-01 00:57:16
1,MS2400E118010101163925,MS,2-400E1,25,CASH,2018-01-01 01:16:39,2018-01-01 01:16:39
2,MS2400E118010101164225,MS,2-400E1,25,CASH,2018-01-01 01:16:42,2018-01-01 01:16:42
3,SSG40018010101274125,SS,G-400,25,CASH,2018-01-01 01:27:41,2018-01-01 01:27:41
4,SSFI4151801010137505,SS,FI-415,5,CASH,2018-01-01 01:37:50,2018-01-01 01:37:50


In [11]:
portal_up = pd.concat([portal, update])
portal_up = portal_up.sort_values(by='trans_start')
portal_up.head()

,uuid,meter_type,pole_id,trans_amt,pay_method,trans_start,meter_expire
0,SSJ50718010100571650,SS,J-507,50,CASH,2018-01-01 00:57:16,2018-01-01 00:57:16
1,MS2400E118010101163925,MS,2-400E1,25,CASH,2018-01-01 01:16:39,2018-01-01 01:16:39
2,MS2400E118010101164225,MS,2-400E1,25,CASH,2018-01-01 01:16:42,2018-01-01 01:16:42
3,SSG40018010101274125,SS,G-400,25,CASH,2018-01-01 01:27:41,2018-01-01 01:27:41
4,SSFI4151801010137505,SS,FI-415,5,CASH,2018-01-01 01:37:50,2018-01-01 01:37:50


In [12]:
dupes = portal_up[portal_up.duplicated(['uuid'], keep=False)]
dupes.shape

(18, 7)

In [13]:
portal_up.shape

(7560646, 7)

In [14]:
portal_up = portal_up.drop_duplicates('uuid', keep='last')

In [15]:
portal_up_18 = portal_up[portal_up['trans_start'] >= '2018-01-01 00:00:00']

In [16]:
portal_up_18.shape

(7560637, 7)

In [17]:
portal_up_18['trans_start'].max()

Timestamp('2018-10-10 22:12:30')

In [18]:
portal_up_18['trans_start'].min()

Timestamp('2018-01-01 00:57:16')

In [19]:
portal_up_18.to_csv('../../../../data/prod/treas_parking_payments_2018_datasd.csv', index=False, encoding='utf-8', doublequote=True, date_format="%Y-%m-%d %H:%M:%S")

In [27]:
portal2 = pd.read_csv('../../../../data/prod/treas_parking_payments_2018_datasd.csv')

In [28]:
portal2.shape

(6796605, 7)

In [29]:
portal2_up = portal2.drop_duplicates('uuid', keep='last')

In [30]:
portal2_up.shape

(6796605, 7)

In [31]:
portal2_up['trans_start'].max()

'2018-09-11 19:58:53'